In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import os

# Settings

In [2]:
def get_settings(tr=2, verbose=True):
    n_runs = 3
    n_sessions = 2
    if tr == 2:
        n_trials = 128
        jitters = [0.5, 1.0, 1.5, 2.0]
        volumes_per_trial = 4
    elif tr == 3:
        n_trials = 114
        jitters = [0.75, 1.5, 2.25, 3]
        volumes_per_trial = 3
    
    n_stop_trials = int(np.ceil(n_trials/4))
    perc = (n_stop_trials / n_trials)*100
    trial_duration = volumes_per_trial*tr
    total_duration = trial_duration*n_trials
    total_duration_min = total_duration/60
    total_volumes = 1 + n_trials*volumes_per_trial
    
    if verbose:
        print('Settings:\n\n\
        Sessions: {n_sessions}\n\
        Runs per session: {n_runs}\n\
        Trials per run: {n_trials}\n\
        Stop trials per run: {n_stop_trials} ({perc:.2f} percent)\n\
        Assuming a TR of {tr} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {tr}*{trial_duration}*{n_trials} = {total_duration} seconds = {total_duration_min} min\n\
        Total number of volumes necessary: 1+{n_trials}*{volumes_per_trial} = {total_volumes} + warm-up pulses'.format(**locals()))
        
    return({'jitter': jitters,
            'n_stop_trials': n_stop_trials,
            'n_trials': n_trials})

In [3]:
get_settings(2)

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses


{'jitter': [0.5, 1.0, 1.5, 2.0], 'n_stop_trials': 32, 'n_trials': 128}

In [4]:
def generate_design(n_trials, jitters, n_stop_trials):
    # make direction array
    
    design = pd.DataFrame({'direction': np.tile([0, 1], int(np.ceil(n_trials/2))),
                           'stop_trial': 0})
    design.loc[:(n_stop_trials-1), 'stop_trial'] = 1
    design = design.sample(frac=1).reset_index(drop=True)
    
    # make jitter array
    design['jitter'] = np.random.choice(jitters, size=n_trials, replace=True)

    return(design)

In [8]:
settings = get_settings(tr=2)
generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses


,direction,stop_trial,jitter
0,1,0,1.0
1,1,1,2.0
2,1,0,2.0
3,1,0,2.0
4,1,0,1.0
5,0,0,1.5
6,0,0,2.0
7,1,0,1.5
8,0,0,1.0
9,1,0,1.5


In [12]:
n_subjects = 20
n_runs = 3

In [13]:
#
save_dir = './designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.arange(1,n_subjects+1):
    for session_tr in range(2,4):
        designs_this_session = []
        
        for run in range(1,n_runs+1):
            settings = get_settings(tr=session_tr)
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design['block'] = run
            
            designs_this_session.append(design)
        
        design = pd.concat(designs_this_session)
        fn = 'sub-' + str(subject_id).zfill(3) + '_tr-' + str(session_tr) + '_design'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 

        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 114
        Stop trials per run: 29 (25.44 percent)
        Assuming a TR of 3 seconds
        Jitter options: [0.75, 1.5, 2.25, 3] seconds
        Total duration: 3*9*114 = 1026 seconds = 17.1 min
        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 114
        Stop trials per run: 29 (25.44 percent)
        Assuming a TR of 3 seconds
        Jitter options: [0.75, 1.5, 2.25, 3] seconds
        Total duration: 3*9*114 = 1026 seconds = 17.1 min
        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
sub-009_tr-3_design
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0

        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 114
        Stop trials per run: 29 (25.44 percent)
        Assuming a TR of 3 seconds
        Jitter options: [0.75, 1.5, 2.25, 3] seconds
        Total duration: 3*9*114 = 1026 seconds = 17.1 min
        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 114
        Stop trials per run: 29 (25.44 percent)
        Assuming a TR of 3 seconds
        Jitter options: [0.75, 1.5, 2.25, 3] seconds
        Total duration: 3*9*114 = 1026 seconds = 17.1 min
        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
sub-017_tr-3_design
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0

### Generate design for practice
Use a single block of tr=2 settings

In [15]:
save_dir = './designs'

settings = get_settings(tr=2)
design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
design['block'] = 1
design = design.iloc[:75]  #only do 75 trials; that takes 10min
design
fn = 'sub-practice_tr-' + str(2) + '_design'
print(fn)
design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
sub-practice_tr-2_design


### Generate design for pilot session
We do 4 runs here, with tr of 3, 2, 3, 2 (corresponding to ME, SE, ME, SE)

In [29]:
designs = []
for block, tr in enumerate([3,2,3,2]):
    settings = get_settings(tr=tr, verbose=False)
    design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
    design['block'] = block+1
    designs.append(design)
design = pd.concat(designs)


fn = 'sub-pilot_tr-' + str(2) + '_design'
print(fn)
design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

sub-pilot_tr-2_design


In [28]:
design

,direction,stop_trial,jitter,block
0,1,0,2.25,1
1,1,0,0.75,1
2,0,1,3.00,1
3,1,0,2.25,1
4,1,0,1.50,1
5,0,0,1.50,1
6,1,0,0.75,1
7,1,0,1.50,1
8,0,0,0.75,1
9,1,0,3.00,1


### For debug?

In [9]:
# for debug
n_runs = 3
n_sessions = 2
n_trials = 10
tr = 3
jitters = [0.75, 1.5, 2.25, 3]
n_stop_trials = int(np.ceil(n_trials/4))

save_dir = './designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in ['DEBUG']:
    for session_tr in range(2,4):
        designs_this_session = []
        
        for run in range(1,n_runs+1):
            settings = get_settings(session_tr)
            design = generate_design(n_trials, settings['jitter'][:n_trials], n_stop_trials)
            design['block'] = run
            design = design.iloc[:n_trials]
            
            designs_this_session.append(design)
        
        design = pd.concat(designs_this_session)
        fn = 'sub-' + str(subject_id).zfill(3) + '_tr-' + str(session_tr) + '_design'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')
design

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 

,direction,stop_trial,jitter,block
0,1,1,2.25,1
1,0,0,1.50,1
2,1,0,2.25,1
3,0,1,2.25,1
4,1,0,0.75,1
5,0,0,1.50,1
6,0,0,0.75,1
7,1,0,0.75,1
8,0,1,1.50,1
9,1,0,1.50,1


In [86]:
design.groupby('direction').stop_trial.sum()

direction
0    6
1    3
Name: stop_trial, dtype: int64

# Finally, create designs for Presentation
This is only meant as a back-up. In case PsychoPy doesn't work out, we can run the experiment in Presentation. The code requires some input files defining, for every run, for every trial, the arrow direction and whether it's a stop trial

These files are tsv saved as txt, with two columns: direction (-1 / 1) and is_stop_trial (0 / 1)

In [32]:
subject_id = 1
subject_id % 2

1

In [42]:
tr_3_folder = './presentation_settings_tr3'
tr_2_folder = './presentation_settings_tr2'
os.makedirs(tr_3_folder)
os.makedirs(tr_2_folder)

for subject_id in range(1, n_subjects+1):
    if subject_id % 2 == 1:  # UNEVEN subjects start with ME, then SE
        settings = get_settings(tr=3)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_3_folder + '/stop_settings_S' + str(subject_id) + 's1r' + str(run) + '.txt', sep='\t', header=False, index=False)
        
        # Do the same but now tr=2
        settings = get_settings(tr=2)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_2_folder + '/stop_settings_S' + str(subject_id) + 's2r' + str(run) + '.txt', sep='\t', header=False, index=False)
    
    elif subject_id % 2 == 0:  # EVEN subjects: start with SE, then ME
        settings = get_settings(tr=2)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_2_folder + '/stop_settings_S' + str(subject_id) + 's1r' + str(run) + '.txt', sep='\t', header=False, index=False)

        settings = get_settings(tr=3)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_3_folder + '/stop_settings_S' + str(subject_id) + 's2r' + str(run) + '.txt', sep='\t', header=False, index=False)

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 114
        Stop trials per run: 29 (25.44 percent)
        Assuming a TR of 3 seconds
        Jitter options: [0.75, 1.5, 2.25, 3] seconds
        Total duration: 3*9*114 = 1026 seconds = 17.1 min
        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 114
        Stop trials per run: 29 (25.44 percent)
        Assuming a TR of 3 seconds
        Jitter options: [0.75, 1.5, 2.25, 3] seconds
        Total duration: 3*9*114 = 1026 seconds = 17.1 min
        Total number of volumes necessary: 1+114*3 = 343 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Stop trials per run: 32 (25.00 percent)
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666

In [41]:
### Load everything to check
import glob
fns = glob.glob('./stop_settings_*.txt')

for fn in fns:
    sub_n = fn.split('_')[-1].split('.')[0]
    dat = pd.read_csv(fn, header=None, sep='\t')
    n_trials = dat.shape[0]
    n_stop_trials = dat.iloc[:,1].sum()
    print('File: %s, n trials: %d, n stop trials: %d' %(sub_n, n_trials, n_stop_trials))

File: S10s1r1, n trials: 128, n stop trials: 32
File: S10s1r2, n trials: 128, n stop trials: 32
File: S10s1r3, n trials: 128, n stop trials: 32
File: S10s2r1, n trials: 114, n stop trials: 29
File: S10s2r2, n trials: 114, n stop trials: 29
File: S10s2r3, n trials: 114, n stop trials: 29
File: S11s1r1, n trials: 114, n stop trials: 29
File: S11s1r2, n trials: 114, n stop trials: 29
File: S11s1r3, n trials: 114, n stop trials: 29
File: S11s2r1, n trials: 128, n stop trials: 32
File: S11s2r2, n trials: 128, n stop trials: 32
File: S11s2r3, n trials: 128, n stop trials: 32
File: S12s1r1, n trials: 128, n stop trials: 32
File: S12s1r2, n trials: 128, n stop trials: 32
File: S12s1r3, n trials: 128, n stop trials: 32
File: S12s2r1, n trials: 114, n stop trials: 29
File: S12s2r2, n trials: 114, n stop trials: 29
File: S12s2r3, n trials: 114, n stop trials: 29
File: S13s1r1, n trials: 114, n stop trials: 29
File: S13s1r2, n trials: 114, n stop trials: 29
File: S13s1r3, n trials: 114, n stop tri

In [6]:
### Some tests
pilot_des = pd.read_csv('./designs/sub-pilot_tr-2_design.csv', sep='\t')

In [11]:
pilot_des.groupby(['block']).jitter.unique()

block
1    [2.25, 0.75, 1.5, 3.0]
2      [1.5, 2.0, 0.5, 1.0]
3    [2.25, 1.5, 3.0, 0.75]
4      [2.0, 1.0, 1.5, 0.5]
Name: jitter, dtype: object